# Data Science Project

Importando las bilbiotecas empleadas

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statistics
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
import seaborn as sb
from sklearn.model_selection import train_test_split

Leyendo el archivo del conjunto de datos filtrando las dos columnas a usar:
- Nationality
- Overall

In [77]:
dataframe = (pd.read_csv("futbol.csv", parse_dates= True)).filter(["", "Age", "Nationality","Club","Value", "Wage",
                    "Preferred Foot", "Position", "Joined", "Contract Valid Until", "Height", "Weight", "Release Clause"])

Visualización del marco de datos importados

In [78]:
dataframe

,ID,Age,Nationality,Club,Value,Wage,Preferred Foot,Position,Joined,Contract Valid Until,Height,Weight,Release Clause
0,158023,31,Argentina,FC Barcelona,€110.5M,€565K,Left,RF,"Jul 1, 2004",2021,5'7,159lbs,€226.5M
1,20801,33,Portugal,Juventus,€77M,€405K,Right,ST,"Jul 10, 2018",2022,6'2,183lbs,€127.1M
2,190871,26,Brazil,Paris Saint-Germain,€118.5M,€290K,Right,LW,"Aug 3, 2017",2022,5'9,150lbs,€228.1M
3,193080,27,Spain,Manchester United,€72M,€260K,Right,GK,"Jul 1, 2011",2020,6'4,168lbs,€138.6M
4,192985,27,Belgium,Manchester City,€102M,€355K,Right,RCM,"Aug 30, 2015",2023,5'11,154lbs,€196.4M
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18202,238813,19,England,Crewe Alexandra,€60K,€1K,Right,CM,"May 3, 2017",2019,5'9,134lbs,€143K
18203,243165,19,Sweden,Trelleborgs FF,€60K,€1K,Right,ST,"Mar 19, 2018",2020,6'3,170lbs,€113K
18204,241638,16,England,Cambridge United,€60K,€1K,Right,ST,"Jul 1, 2017",2021,5'8,148lbs,€165K
18205,246268,17,England,Tranmere Rovers,€60K,€1K,Right,RW,"Apr 24, 2018",2019,5'10,154lbs,€143K


Visualizar cantidad de datos nulos por columna

In [124]:
pd.DataFrame(dataframe.isnull().sum(), columns=['Datos nulos'])

,Datos nulos
ID,0
Age,0
Nationality,0
Club,241
Value,0
Wage,0
Preferred Foot,48
Position,60
Joined,1553
Contract Valid Until,289


Visualizar los datos nulos de acuerdo con la columna Club

In [101]:
(dataframe.loc[dataframe['Club'].isnull()])

,ID,Age,Nationality,Club,Value,Wage,Preferred Foot,Position,Joined,Contract Valid Until,Height,Weight,Release Clause
452,207439,24,Argentina,NaN,€0,€0,Right,CM,NaN,NaN,5'11,165lbs,NaN
538,156713,33,Sweden,NaN,€0,€0,Right,LCB,NaN,NaN,6'4,185lbs,NaN
568,229909,26,Russia,NaN,€0,€0,Right,GK,NaN,NaN,6'2,176lbs,NaN
677,187347,29,Russia,NaN,€0,€0,Right,RB,NaN,NaN,5'10,154lbs,NaN
874,187607,29,Russia,NaN,€0,€0,Right,ST,NaN,NaN,6'5,201lbs,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17197,245164,21,India,NaN,€0,€0,Right,GK,NaN,NaN,6'2,176lbs,NaN
17215,243131,26,Finland,NaN,€0,€0,Right,RB,NaN,NaN,6'4,187lbs,NaN
17339,233526,23,India,NaN,€0,€0,Right,NaN,NaN,NaN,5'9,143lbs,NaN
17436,236452,20,India,NaN,€0,€0,Right,NaN,NaN,NaN,6'0,168lbs,NaN


Visualizar cuáles ID de jugadores no tienen sueldo

In [99]:
(dataframe.loc[dataframe['Wage']== '€0']["ID"])

452      207439
538      156713
568      229909
677      187347
874      187607
          ...  
17197    245164
17215    243131
17339    233526
17436    236452
17539    234508
Name: ID, Length: 241, dtype: int64

Visualizar cuáles ID de jugadores no tienen valor

In [106]:
(dataframe.loc[dataframe['Value']== '€0']["ID"])

452      207439
538      156713
568      229909
677      187347
864      153260
          ...  
17539    234508
17672    237744
17726     51963
17752    100899
18183     53748
Name: ID, Length: 252, dtype: int64

Comprobar que los que no tienen club son los mismo que no tienen sueldo

In [102]:
((dataframe.loc[dataframe['Wage']== '€0']["ID"]) == (dataframe.loc[dataframe['Club'].isnull()]["ID"])).all()

True

Se observa que la cantidad de datos nulos que no tiene pierna preferida es igual a la variable de altura y peso

In [127]:
((dataframe.loc[dataframe['Preferred Foot'].isnull()]["ID"]) == (dataframe.loc[dataframe['Height'].isnull()]["ID"])).all()

True

Lista de actividades:
- Determinar que los de preferred foot son los mismos a height y ver si son los mismos a weight
- Se ha determinado que club y wage son iguales y partiendo de la premisa que son los mismos:
- Determinar si esos 241 pertenecen a los de Joined
- Determinar si esos 241 pertenecen a los de Release clause
- Determinar si esos 241 pertenecen a los de contract valid untl
- Posiblemente agregar al dataframe la columna de loaned from porque parece ser que los que tienen ese campo, no tienen Joined
    ni cláusula de recisión.
- Si se cumple eso, posiblemente cambiar joined a una variable boolean de Loand para indicar si está en préstamo y joined la falta de los de club con los de loaned
- Explicar que se haría el cambio para emplear la técnica de machine learning
- Expresar las razones de porqué entonces eliminar released clause, ya que sólo si no estás en préstamo no tendrá el jugador dicho campo
- Determinar que el tipo de dato sea correcto para el análisis
- Cambiar weight, height, value y wage para que sean de tipo float
- crear nuevo boolean field que indique si está en préstamo o no

In [105]:
((dataframe.loc[dataframe['Release Clause'].isnull()]["ID"]))

28       198710
38       167664
91       187961
166      212523
176      207410
          ...  
17979    241341
18026    241444
18031    243506
18056    244872
18183     53748
Name: ID, Length: 1564, dtype: int64

Comprobación si existe algún dato nulo en el marco de datos

Comprobación del tipo de dato de cada columna del marco de datos

In [5]:
pd.DataFrame(dataframe.dtypes, columns=['Tipo de dato'])

,Tipo de dato
Nationality,object
Overall,int64


Convertimos los valores del conjunto de datos en una matriz de Numpy